In [1]:
# Imports
import os
import bz2
import numpy as np
from PIL import TiffImagePlugin
from IPython.core.display import display
from PIL import Image as Img

In [2]:
import sys
sys.path.append("../")
from algorithms.flow import flow
from data_structures.location_graph import LocationGraph

In [3]:
def load_image_data() -> TiffImagePlugin.TiffImageFile:
    path = os.path.dirname(os.path.abspath('')) + '/data/'
    data = bz2.decompress(open(path + 'dem.tif.bz2', 'rb').read())
    with open(path + 'dem2.tiff', 'wb') as handle:
        handle.write(data)
    return Img.open(path + 'dem2.tiff')

In [4]:
def load_and_crop_data():
    raw_height_array = np.asarray(load_image_data())
    crop_size = 2000
    offset = 0
    cropped = raw_height_array[offset:offset+crop_size, offset:offset+crop_size]
    print(type(cropped))
    print(cropped.max())
    print(cropped.min())
    return cropped
cropped_data = load_and_crop_data()

<class 'numpy.ndarray'>
1574
145


In [5]:
%%time
height_map = cropped_data.tolist()
graph = LocationGraph(height_map)

CPU times: user 1min 31s, sys: 2.99 s, total: 1min 34s
Wall time: 1min 33s


In [6]:
def normalize_values(i):
    return int((i-1)*255/1193)

In [17]:
class ImageWriter(object):
    def __init__(self, path, size, save_frequency=10000):
        if not os.path.exists(path):
            os.makedirs(path)
        self.write_path = path
        self.index = 0
        self.image = Img.new('L', (size, size))
        self.pixels = self.image.load()
        self.save_frequency=save_frequency
        self.image_number = 0
        self.size = size
        self.previous_percent = 0

    def update(self, node):
        self.index += 1
        self.pixels[node.home[0],node.home[1]] = normalize_values(node.flow)
        if self.index % self.save_frequency == 0:
            filename = f"{self.write_path}{self.image_number}.tiff"
            (self.image).save(filename)
            
            percent_finished = int(self.index*100/self.size**2)
            if percent_finished != self.previous_percent:
                print(f'Processed {percent_finished}% of {self.size**2} images.')
                self.previous_percent = percent_finished
            self.image_number += 1

# Initialize image file writer
def initialize_writer():
    writer_path = f"{os.path.dirname(os.path.dirname(os.path.abspath('')))}/data/flows/"
    print(f"Writer will save images to {writer_path}")
    writer = ImageWriter(writer_path, 2000, 2500)
    return writer
writer = initialize_writer()

Writer will save images to /home/andrew/personal/river-flow/data/flows/


In [18]:
%%time
nodes_with_flow = flow(graph, writer)

Processed 1% of 4000000 images.
Processed 2% of 4000000 images.
Processed 3% of 4000000 images.
Processed 4% of 4000000 images.
Processed 5% of 4000000 images.
Processed 6% of 4000000 images.
Processed 7% of 4000000 images.
Processed 8% of 4000000 images.
Processed 9% of 4000000 images.
Processed 10% of 4000000 images.
Processed 11% of 4000000 images.
Processed 12% of 4000000 images.
Processed 13% of 4000000 images.
Processed 14% of 4000000 images.
Processed 15% of 4000000 images.
Processed 16% of 4000000 images.
Processed 17% of 4000000 images.
Processed 18% of 4000000 images.
Processed 19% of 4000000 images.
Processed 20% of 4000000 images.
Processed 21% of 4000000 images.
Processed 22% of 4000000 images.
Processed 23% of 4000000 images.
Processed 24% of 4000000 images.
Processed 25% of 4000000 images.
Processed 26% of 4000000 images.
Processed 27% of 4000000 images.
Processed 28% of 4000000 images.
Processed 29% of 4000000 images.
Processed 30% of 4000000 images.
Processed 31% of 40

In [19]:
# flows = [n.flow for n in nodes_with_flow.ascending()]
flows = [n.flow for n in nodes_with_flow.ascending()]
print(len(flows))
print(max(flows))
print(min(flows))

4000000
615404258.500778
4.0
